In [ ]:
import cv2 as cv
# from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import ipywidgets as widgets

import numpy as np

In [ ]:
img = cv.imread('./lab_01.jpg')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
(img_height, img_width, _) = img.shape
img_ratio = 1000 / img_width
img = cv.resize(img, (1000, int(img_height * img_ratio)), interpolation=cv.INTER_AREA)

In [ ]:
plt.imshow(img)

In [ ]:
hsv_img = cv.cvtColor(img, cv.COLOR_RGB2HSV)

In [ ]:
def masker(mask):
    kernel = np.ones((5,5),np.uint8)
    mask2 = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    mask3 = cv.morphologyEx(mask2, cv.MORPH_CLOSE, kernel)
    masks = np.hstack((mask, mask2, mask3))
    return masks

In [ ]:
_mask = cv.inRange(
    hsv_img,
    (100, 0, 200),
    (150, 50, 255)
)

In [ ]:
masked = masker(_mask)

In [ ]:
plt.imshow(_mask)

In [ ]:
AREA_TOP = (0, 100)
AREA_BOTTOM = (350, 450) # based on ratio
AREA_LEFT = (200, 300)
AREA_RIGHT = (550, 650)

ORIGIN_TOP_LEFT = (0, 0)
ORIGIN_TOP_RIGHT = (0, 199)
ORIGIN_BOTTOM_LEFT = (199, 0)
ORIGIN_BOTTOM_RIGHT = (199, 199)

def roi_corner(mask, area_x, area_y, origin):
    roi_mask = mask[area_x[0]:area_x[1], area_y[0]:area_y[1]]
    roi_mask = np.float32(roi_mask)
    roi_corners = cv.cornerHarris(roi_mask, blockSize=3, ksize=3, k=0.05)
    
    roi_points = np.argwhere(roi_corners > 0.025 * roi_corners.max())

    roi_distances = np.array([np.linalg.norm(origin - p) for p in roi_points])
    roi_index = np.argmin(roi_distances)
    roi_point = roi_points[roi_index]

    return (roi_point[0] + area_x[0], roi_point[1] + area_y[0])

In [ ]:
top_left_corner = roi_corner(_mask, AREA_TOP, AREA_LEFT, ORIGIN_TOP_LEFT)
img[top_left_corner[0]-10:top_left_corner[0]+10, top_left_corner[1]-10:top_left_corner[1]+10] = [255, 0, 0]

In [ ]:
top_right_corner = roi_corner(_mask, AREA_TOP, AREA_RIGHT, ORIGIN_TOP_RIGHT)
img[top_right_corner[0]-10:top_right_corner[0]+10, top_right_corner[1]-10:top_right_corner[1]+10] = [255, 0, 0]

In [ ]:
bottom_left_corner = roi_corner(_mask, AREA_BOTTOM, AREA_LEFT, ORIGIN_BOTTOM_LEFT)
img[bottom_left_corner[0]-10:bottom_left_corner[0]+10, bottom_left_corner[1]-10:bottom_left_corner[1]+10] = [255, 0, 0]

In [ ]:
bottom_right_corner = roi_corner(_mask, AREA_BOTTOM, AREA_RIGHT, ORIGIN_BOTTOM_RIGHT)
img[bottom_right_corner[0]-10:bottom_right_corner[0]+10, bottom_right_corner[1]-10:bottom_right_corner[1]+10] = [255, 0, 0]

In [ ]:
plt.imshow(img)

In [ ]:
old_perspective_points = np.float32([top_left_corner, bottom_left_corner, top_right_corner, bottom_right_corner])
new_perspective_points = np.float32([[0, 0], [300, 0], [0, 300], [300, 300]])

In [ ]:
perspective_matrix = cv.getPerspectiveTransform(old_perspective_points, new_perspective_points)

In [ ]:
height, width = img.shape[:2]
fixed_perspective_img = cv.warpPerspective(img, perspective_matrix, (300, 300))

In [ ]:
plt.imshow(fixed_perspective_img)